In [20]:
import numpy as np
import pandas as pd

from xml.sax import ContentHandler, parse

# https://stackoverflow.com/questions/33470130/read-excel-xml-xls-file-with-pandas
# Reference https://goo.gl/KaOBG3
class ExcelHandler(ContentHandler):
    def __init__(self):
        self.chars = [  ]
        self.cells = [  ]
        self.rows = [  ]
        self.tables = [  ]
    def characters(self, content):
        self.chars.append(content)
    def startElement(self, name, atts):
        if name=="Cell":
            self.chars = [  ]
        elif name=="Row":
            self.cells=[  ]
        elif name=="Table":
            self.rows = [  ]
    def endElement(self, name):
        if name=="Cell":
            self.cells.append(''.join(self.chars))
        elif name=="Row":
            self.rows.append(self.cells)
        elif name=="Table":
            self.tables.append(self.rows)

excelHandler = ExcelHandler()
parse('TrackingResults_cut.xml', excelHandler)

# the index must be adjusted so that it finds the correct row
FIRST_ROW = 46
print(excelHandler.tables[0][FIRST_ROW])

df1 = pd.DataFrame(excelHandler.tables[0][FIRST_ROW+1:], columns=excelHandler.tables[0][FIRST_ROW])

df1 = df1.dropna()
df1 = df1.dropna(axis=1, how='all')
df1 = df1.drop(columns = [''])
#df1['X'] = pd.to_numeric(df1['X'])
#df1['y'] = pd.to_numeric(df1['y'])
df1['Cell ID'] = df1['Cell ID'].astype('int')
df1['Time index'] = df1['Time index'].astype('int')
df1['X'] = df1['X'].astype('float')
df1['y'] = df1['y'].astype('float')

print(df1.columns)


['Cell ID', 'Image name', 'Time (sec)', 'Time index', 'X', 'y', 'Distance from previous image [pixels]', 'Speed between two images [pixels/h]', '']
Index(['Cell ID', 'Image name', 'Time (sec)', 'Time index', 'X', 'y',
       'Distance from previous image [pixels]',
       'Speed between two images [pixels/h]'],
      dtype='object')


In [25]:
df2 = df1.drop(columns=['Image name','Time (sec)','Distance from previous image [pixels]','Speed between two images [pixels/h]'])
df2['nn'] = -1
df2['d'] = -1

from sklearn.neighbors import NearestNeighbors

# loop over time points
for t in range(1,max(df2['Time index'] + 1)):
    print('t: ' + str(t))

    # create subset of the original data at time t
    is_t = df2['Time index'] == t
    df2_at_t = df2[is_t].copy()
    # index of the data at time t
    df2_at_t['index_at_t'] = range(0,len(df2_at_t.index))
    print('dt2 at t')
    #print(df2_at_t[:3])
    print(df2_at_t)
    
    # find the nearest neighbors at timepoint t.
    xy = np.array(df2_at_t[['X','y']].as_matrix())
    #print(xy)
    nbrs = NearestNeighbors(n_neighbors=2, algorithm='brute').fit(xy)
    distances, indices = nbrs.kneighbors(xy)
    #print('distances...')
    #print(distances[1])
    print('indices...')
    print(indices[:,1])
    
    # nearest neighbors expressed with index at time t
    df2_at_t['nn_at_t'] = indices[:,1]

    # create mapping from index at time t to original cell id
    # https://stackoverflow.com/questions/18012505/python-pandas-dataframe-columns-convert-to-dict-key-and-value
    cell_id_dict = dict(zip(df2_at_t['index_at_t'], df2_at_t['Cell ID']))
    print('map from index at t to cell id:')
    print(cell_id_dict)
    
    # use the mapping to find out cell id of the neighbors, add a new column
    # https://stackoverflow.com/questions/18453566/python-dictionary-get-list-of-values-for-list-of-keys
    #df2_at_t['nn'] = [cell_id_dict[x] for x in df2_at_t['nn_at_t']]
    df2_at_t['nn'] = list(map(cell_id_dict.__getitem__, df2_at_t['nn_at_t']))
    
    # add distance column
    df2_at_t['d'] = distances[:,1]
    print('dt2 at t')
    #print(df2_at_t[:3])
    print(df2_at_t)
    
    print('merge...')
    # pd.merge does not work because after the first timepoint all rows will have
    # nn and d set (NaN if not a real result from timepoint 1).
    #cols_to_use = df2_at_t.columns.difference(df2.columns)
    #print(cols_to_use)
    #df2 = pd.merge(df2, df2_at_t[cols_to_use], left_index=True, right_index=True, how='outer')
    
    # https://stackoverflow.com/questions/46807354/merge-2-columns-with-priority
    df2['nn'] = df2_at_t['nn'].combine_first(df2['nn'])
    df2['d'] = df2_at_t['d'].combine_first(df2['d'])
    #print(df2[:3])
    print(df2)

print(df2)

t: 1
dt2 at t
   Cell ID  Time index      X      y  nn  d  index_at_t
0        1           1   39.0  105.0  -1 -1           0
2        2           1  163.0  218.0  -1 -1           1
4        3           1  471.0  262.0  -1 -1           2
indices...
[1 0 1]
map from index at t to cell id:
{0: 1, 1: 2, 2: 3}
dt2 at t
   Cell ID  Time index      X      y  nn           d  index_at_t  nn_at_t
0        1           1   39.0  105.0   2  167.764716           0        1
2        2           1  163.0  218.0   1  167.764716           1        0
4        3           1  471.0  262.0   2  311.126984           2        1
merge...
   Cell ID  Time index      X      y   nn           d
0        1           1   39.0  105.0  2.0  167.764716
1        1           2   39.0  118.0 -1.0   -1.000000
2        2           1  163.0  218.0  1.0  167.764716
3        2           2  152.0  205.0 -1.0   -1.000000
4        3           1  471.0  262.0  2.0  311.126984
5        3           2  465.0  262.0 -1.0   -1.000000


/home/hajaalin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [26]:
import math
np.sqrt((163-39)**2 + (218-105)**2)
#np.sqrt((163-103)**2 + (218-360)**2)

167.7647161950331